# *Naan Mudhalvan  Project*
# *Generative AI*
# *Project Title - Sentimental Analysis Using LSTM*
#*Register number-211521243099*
# *Name - I. Mohamed Sameer*

In [ ]:
pip install Keras-Preprocessing


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import re
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score
import math
import nltk

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# Read data
data = pd.read_csv('/content/drive/MyDrive/generative ai naan mudhalvan/IMDB Dataset.csv')
# i have added the dataset in my repository use it since i use google colab i mounted my dataset in my drive

In [8]:
# Define function to remove tags from text
def remove_tags(string):
    removelist = ""
    result = re.sub('<.*?>', '', string)  # Remove HTML tags
    result = re.sub('https://.*', '', result)  # Remove URLs
    result = re.sub(r'[^w'+removelist+']', ' ', result)  # Remove non-alphanumeric characters
    result = result.lower()
    return result

In [9]:
# Apply preprocessing to 'review' column
data['review'] = data['review'].apply(lambda cw : remove_tags(cw))


In [10]:
# Further preprocessing: remove stopwords and lemmatize
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
data['review'] = data['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
data['review'] = data.review.apply(lemmatize_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# Calculate average length of each review
s = sum(len(review.split()) for review in data['review'])
average_length = s / len(data['review'])
print("Average length of each review:", average_length)

Average length of each review: 18.7137


In [12]:
# Percentage of positive and negative reviews
pos_percentage = (data['sentiment'].value_counts(normalize=True)['positive']) * 100
neg_percentage = (data['sentiment'].value_counts(normalize=True)['negative']) * 100
print("Percentage of reviews with positive sentiment:", pos_percentage, "%")
print("Percentage of reviews with negative sentiment:", neg_percentage, "%")

Percentage of reviews with positive sentiment: 50.0 %
Percentage of reviews with negative sentiment: 50.0 %


In [13]:
# Prepare data for training
reviews = data['review'].values
labels = data['sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify=encoded_labels)

In [14]:
# Hyperparameters
vocab_size = 3000
oov_tok = ''
embedding_dim = 100
max_length = 200
padding_type = 'post'
trunc_type = 'post'

In [15]:
# Tokenize sentences
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index


In [16]:
# Convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)


In [17]:
# Convert test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)


In [18]:
# Model definition
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [19]:
# Compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Model summary
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 200, 100)          300000    
                                                                 
 bidirectional_2 (Bidirecti  (None, 128)               84480     
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 24)                3096      
                                                                 
 dense_5 (Dense)             (None, 1)                 25        
                                                                 
Total params: 387601 (1.48 MB)
Trainable params: 387601 (1.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Model training
num_epochs = 5
history = model.fit(train_padded, train_labels, epochs=num_epochs, verbose=1, validation_split=0.1)
# since epoch takes high time
#add higher number of epoch for better  prediction

Epoch 1/5
1055/1055 [==============================] - 253s 235ms/step - loss: 0.6931 - accuracy: 0.5007 - val_loss: 0.6926 - val_accuracy: 0.5189
Epoch 2/5
1055/1055 [==============================] - 246s 233ms/step - loss: 0.6928 - accuracy: 0.5101 - val_loss: 0.6921 - val_accuracy: 0.5272
Epoch 3/5
1055/1055 [==============================] - 281s 266ms/step - loss: 0.6926 - accuracy: 0.5129 - val_loss: 0.6939 - val_accuracy: 0.5133
Epoch 4/5
1055/1055 [==============================] - 241s 229ms/step - loss: 0.6925 - accuracy: 0.5153 - val_loss: 0.6921 - val_accuracy: 0.5163
Epoch 5/5
1055/1055 [==============================] - 242s 229ms/step - loss: 0.6924 - accuracy: 0.5157 - val_loss: 0.6925 - val_accuracy: 0.5144


In [21]:
# Model evaluation
test_loss, test_accuracy = model.evaluate(test_padded, test_labels)
print("Test Accuracy:", test_accuracy)


391/391 [==============================] - 21s 55ms/step - loss: 0.6926 - accuracy: 0.5117
Test Accuracy: 0.511680006980896


In [27]:
# Sentiment prediction for custom sentences
# In the below sentence list  you can give your own sentence also to check the model performance
sentence = ["The movie was very touching and heartwarming",
            "I have never seen a terrible movie like this",
            "The movie plot is terrible but it had good acting",
            "waste of time for watching"]

sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, padding=padding_type, maxlen=max_length)

predictions = model.predict(padded)

for i in range(len(sentence)):
    print(sentence[i])
    if predictions[i] >= 0.5:
        print("Predicted sentiment: Positive")
    else:
        print("Predicted sentiment: Negative")

1/1 [==============================] - 0s 40ms/step
The movie was very touching and heartwarming
Predicted sentiment: Positive
I have never seen a terrible movie like this
Predicted sentiment: Positive
The movie plot is terrible but it had good acting
Predicted sentiment: Positive
waste of time for watching
Predicted sentiment: Positive
